Task6 模型融合方式任意，并结合Task5给出你的最优结果

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import matplotlib as mpl
from sklearn.model_selection import train_test_split

In [2]:
#正常显示中文
mpl.rcParams['font.sans-serif']=[u'SimHei']
mpl.rcParams['axes.unicode_minus'] = False
#用于抑制第三方警告
warnings.filterwarnings("ignore")

In [3]:
pd.options.display.max_rows = 10   #最大显示10行
data = pd.read_csv(r"D:\datasets\data.csv",encoding = "gbk")  #读取数据  
data_1 = data.copy()
data_1.describe()

,Unnamed: 0,custid,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
count,4754.000000,4.754000e+03,4752.000000,4752.000000,4754.000000,4751.000000,4752.000000,4752.000000,4752.000000,4746.000000,...,4457.000000,4457.000000,4457.000000,4457.000000,4457.000000,4457.000000,4457.000000,4457.000000,4450.000000,4457.000000
mean,6008.414178,1.690993e+06,0.021806,0.901294,1940.197728,14.160674,0.804411,0.365425,17.502946,29.029920,...,3390.038142,1820.357864,9187.009199,76.042630,4.732331,5.227507,16153.690823,8007.696881,24.112809,55.181512
std,3452.071428,1.034235e+06,0.041527,0.144856,3923.971494,694.180473,0.196920,0.170196,4.475616,22.722432,...,1474.206546,583.418291,7371.257043,14.536819,2.974596,3.409292,14301.037628,5679.418585,37.725724,53.486408
min,5.000000,1.140000e+02,0.000000,0.000000,0.000000,0.000000,0.120000,0.033000,2.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.000000,-2.000000
25%,3106.000000,7.593358e+05,0.010000,0.880000,0.000000,0.615000,0.670000,0.233000,15.000000,16.000000,...,2300.000000,1535.000000,4800.000000,77.000000,2.000000,3.000000,7800.000000,4737.000000,5.000000,10.000000
50%,6006.500000,1.634942e+06,0.010000,0.960000,500.000000,0.970000,0.860000,0.350000,17.000000,23.000000,...,3100.000000,1810.000000,7700.000000,79.000000,4.000000,5.000000,13800.000000,7050.000000,14.000000,36.000000
75%,8999.000000,2.597905e+06,0.020000,0.990000,2000.000000,1.600000,1.000000,0.480000,20.000000,32.000000,...,4300.000000,2100.000000,11700.000000,80.000000,7.000000,7.000000,20400.000000,10000.000000,24.000000,91.000000
max,11992.000000,4.004694e+06,1.000000,1.000000,68000.000000,47596.740000,1.000000,0.941000,42.000000,285.000000,...,10000.000000,6900.000000,87100.000000,87.000000,18.000000,20.000000,266400.000000,82800.000000,360.000000,323.000000


In [5]:
#将空值替换为np.nan
for columns in data_1.columns:
    data_1.loc[data_1[columns] == "NA",columns] = np.nan 
    
#删除无效特征
Irrelevant_features = ['Unnamed: 0', 'custid', 'trade_no', 'bank_card_no', 'source', 'id_name', 'latest_query_time','loans_latest_time']
data_2 = data_1.drop(Irrelevant_features,axis = 1)


#根据reg_preference_for_trad删除空值
data_3 = data_2.dropna(subset = ["reg_preference_for_trad"])
data_3 = data_2.dropna(subset = ["status"])

#在指定阈值的前提下对特征进行删除 
data_3 = data_3.dropna(axis = 1,thresh = 1000)
data_3 = data_3.dropna(axis = 0,thresh = 75)

#对空值采用均值进行填充
data_3 = data_3.fillna(data_3.mean())
print(data_3.shape)

(4423, 82)


In [6]:
#去除标签
data_non_labels = data_3.drop("status",axis = 1)

#采样标签
data_labels = data_3["status"]

#切分数据集
X_train_set,X_test_set,y_train_set,y_test_set = train_test_split(data_non_labels,data_labels,test_size = 0.3,random_state = 2018)
print(X_train_set.shape)

#取reg_preference_for_trad
reg_preference_for_trad = X_train_set["reg_preference_for_trad"].copy()
print(reg_preference_for_trad.shape)

#去除reg_preference_for_trad
data_non_reg_preference = X_train_set.drop("reg_preference_for_trad",axis = 1)
print(data_non_reg_preference.shape)

(3096, 81)
(3096,)
(3096, 80)


In [7]:
from sklearn.feature_selection import VarianceThreshold
threshold = 10
vt = VarianceThreshold(threshold = threshold)
data_non_reg_preference = vt.fit_transform(data_non_reg_preference)
data_non_reg_preference.shape

(3096, 58)

In [8]:
#对数据进行归一化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data_non_reg_preference_scalered = scaler.fit_transform(data_non_reg_preference)
print(data_non_reg_preference_scalered.shape)

(3096, 58)


In [9]:
#尝试着将城市信息转变为onehot矩阵  
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
encoder = LabelEncoder().fit_transform(reg_preference_for_trad)
reg_preference_encodered =  OneHotEncoder().fit_transform(encoder.reshape(-1,1)).toarray()
print(reg_preference_encodered.shape)

#将reg_preference_for_trad再添加进去
data_prepared = np.hstack((data_non_reg_preference_scalered,reg_preference_encodered))
print(data_prepared.shape)

(3096, 5)
(3096, 63)


In [10]:
data_labels = y_train_set

print(data_prepared.shape)
print(data_labels.shape)

(3096, 63)
(3096,)


采用Voting进行集成

In [11]:
#采用Voting进行硬投票

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score,recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve     
from sklearn.metrics import mean_squared_error  #采用均方误差进行评价
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

best_log = LogisticRegression(C=4, class_weight='balanced', dual=False, fit_intercept=True,intercept_scaling=1,
                              l1_ratio=None, max_iter=100,multi_class='warn', n_jobs=-1, penalty='l2',
                              random_state=2018, solver='sag', tol=0.0001, verbose=0,warm_start=False)
best_rnf_clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',max_depth=11, max_features=12, 
                                          max_leaf_nodes=28,min_impurity_decrease=0.0, min_impurity_split=None,
                                          min_samples_leaf=1, min_samples_split=2,min_weight_fraction_leaf=0.0, n_estimators=20,
                                          n_jobs=None, oob_score=False, random_state=2018,verbose=0, warm_start=False)
svc_clf = SVC(C=3, cache_size=200, class_weight=None, coef0=0.0,decision_function_shape='ovr',
          degree=3, gamma="auto", kernel='linear',max_iter=-1, probability=False, random_state=None, 
          shrinking=True,tol=0.001, verbose=False)

voting_clf = VotingClassifier(estimators = [("best_log",best_log),("svc_clf",svc_clf),("best_rnf_clf",best_rnf_clf)],voting = "hard")



voting_clf.fit(data_prepared,data_labels)
voting_prediction = voting_clf.predict(data_prepared)

print(accuracy_score(data_labels,voting_prediction))

0.8104005167958657


采用baggingclassifier进行评价

In [21]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bagging_clf = BaggingClassifier(DecisionTreeClassifier(class_weight='balanced', criterion='entropy',max_depth=5, max_features=8,
                                                       max_leaf_nodes=22,min_impurity_decrease=0.0, min_impurity_split=None,
                                                       min_samples_leaf=1, min_samples_split=14,min_weight_fraction_leaf=0.0,
                                                       presort=False,random_state=2018, splitter='best'),n_estimators = 500,
                                max_samples = 1500,bootstrap = True,n_jobs = -1,oob_score = True)
bagging_clf.fit(data_prepared,data_labels)
bagging_prediction = bagging_clf.predict(data_prepared)
print(accuracy_score(data_labels,bagging_prediction))

0.7932816537467701


In [16]:
bagging_clf.oob_score_

0.7487080103359173

In [18]:
from sklearn.ensemble import RandomForestClassifier
rnf_clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',max_depth=11, max_features=12, 
                                          max_leaf_nodes=28,min_impurity_decrease=0.0, min_impurity_split=None,
                                          min_samples_leaf=1, min_samples_split=2,min_weight_fraction_leaf=0.0, n_estimators=200,
                                          n_jobs=None, oob_score=True, random_state=2018,verbose=0, warm_start=False)

rnf_clf.fit(data_prepared,data_labels)
rnf_prediction = rnf_clf.predict(data_prepared)
print(accuracy_score(data_labels,rnf_prediction))

0.8152454780361758


In [19]:
rnf_clf.oob_score_

0.7900516795865633

采用AdaBoost进行评价

In [26]:
from sklearn.ensemble import AdaBoostClassifier
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(class_weight='balanced', criterion='entropy',max_depth=5, max_features=8,
                                                       max_leaf_nodes=22,min_impurity_decrease=0.0, min_impurity_split=None,
                                                       min_samples_leaf=1, min_samples_split=14,min_weight_fraction_leaf=0.0,
                                                       presort=False,random_state=2018, splitter='best'),n_estimators = 30,
                             algorithm = "SAMME.R",learning_rate = 0.5)
ada_clf.fit(data_prepared,data_labels)
ada_prediction = ada_clf.predict(data_prepared) 
print(accuracy_score(data_labels,ada_prediction))

0.9463824289405685


In [31]:
from sklearn.ensemble import GradientBoostingClassifier
gbrt = GradientBoostingClassifier(max_depth = 3,n_estimators = 60,learning_rate = 0.5)
gbrt.fit(data_prepared,data_labels)
gbrt_prediction = gbrt.predict(data_prepared)
print(accuracy_score(data_labels,gbrt_prediction))

0.9189276485788114


In [60]:
from sklearn.model_selection import train_test_split
gbrt_1 = GradientBoostingClassifier(max_depth = 2,n_estimators = 120,learning_rate = 0.05,random_state = 2018)

X_train,X_test,y_train,y_test = train_test_split(data_prepared,data_labels,test_size = 0.2)
gbrt_1.fit(X_train,y_train)
errors = [accuracy_score(y_test,y_pred) for y_pred in gbrt_1.staged_predict(X_test)]
best_estimators = np.argmin(errors)
best_estimators

0

In [63]:
best_gbrt = GradientBoostingClassifier(max_depth = 3,n_estimators = 40,learning_rate = 0.05)
best_gbrt.fit(X_train,y_train)
best_gbrt_prediction = best_gbrt.predict(X_test)
print(accuracy_score(y_test,best_gbrt_prediction))

0.7774193548387097
